In [1]:
import os

os.chdir('/content/drive/MyDrive/Colab Notebooks/Nhan dien bien so Wpod-NET/final-alpr-unconstrained-master')
print(os.getcwd())

/content/drive/MyDrive/Colab Notebooks/Nhan dien bien so Wpod-NET/final-alpr-unconstrained-master


In [2]:
import sys
import numpy as np
import cv2
import argparse
import tensorflow.keras as keras
import matplotlib.pyplot as plt

from random import choice
from os.path import isfile, isdir, basename, splitext
from os import makedirs

from src.keras_utils import save_model, load_model
from src.label import readShapes
from src.loss import loss
from src.utils import image_files_from_folder, show
from src.sampler import augment_sample, labels2output_map
from src.data_generator import DataGenerator

from pdb import set_trace as pause

In [3]:
def load_network(modelpath, input_dim):
    '''
    Hàm này load model và test luôn model xem nó có chạy đúng ko (có đúng là input shape > output shape 16 lần ko)
    '''
    model = load_model(modelpath)  # load eccv model theo mac dinh
    input_shape = (input_dim, input_dim, 3)  # cái này sẽ là (208,208,3)

    # Fixed input size for training
    inputs = keras.layers.Input(shape=(input_dim, input_dim, 3))
    outputs = model(inputs)

    output_shape = tuple([s for s in outputs.shape[1:]])
    output_dim = output_shape[1]
    model_stride = input_dim / output_dim

    assert input_dim % output_dim == 0, \
        'The output resolution must be divisible by the input resolution'

    assert model_stride == 2**4, \
        'Make sure your model generates a feature map with resolution ' \
        '16x smaller than the input'

    return model, model_stride, input_shape, output_shape


def process_data_item(data_item, dim, model_stride):
    # XX là ảnh đã augment
    # pts là 4 tọa độ của 4 đỉnh bounnding box đã chỉnh sửa theo ảnh augment. pts trong range(0,1)
    # llp là đối tượng Label được dựng lên từ pts thôi. thực ra label chính là pts mà
    # data_item[1].pts là dưới dạng tương đối
    XX, llp, pts = augment_sample(data_item[0], data_item[1].pts, dim) ## XX là (208,208,3)
    YY = labels2output_map(data_item[1].lp_type, llp, pts, dim, model_stride) ## YY là (13,13,10)  #### fix
    return XX, YY  # đây chính là cái mình dùng để train đây này.


def load_Data(data_dir):
    Files = image_files_from_folder(data_dir)  # Files là 1 list các img path

    Data = []
    for file in Files:
        # labfile là label file. File label của 1 ảnh có tên giống nó chỉ khác đuôi là txt
        labfile = splitext(file)[0] + '.txt'
        if isfile(labfile):
            L = readShapes(labfile)
            I = cv2.imread(file)
            Data.append([I, L[0]])  # I là ảnh, L[0] là label cho 4 góc của LP
            # L[0] thuộc class Shape()

    return Data


def process_Data(data, xshape=(208, 208, 3), yshape=(13, 13, 10)):
    X = np.empty((len(data),) + xshape)
    Y = np.empty((len(data),) + yshape)

    for (i, data_item) in enumerate(data):
        X[i], Y[i] = process_data_item(data_item, 208, 16)

    return X, Y


def plot_losses(train_loss, val_loss):
    plt.plot(train_loss)
    plt.plot(val_loss)
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('iterations on batch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [4]:
# lấy giá trị các tham số từ command line
netname = 'my_trained_model'
train_dir = '/content/drive/MyDrive/Colab Notebooks/Nhan dien bien so Wpod-NET/final-alpr-unconstrained-master/samples/train_resized'
outdir = 'models/my-trained-model/'

iterations = 10000
batch_size = 64
dim = 208  # input_shape là (208,208,3)

In [5]:
if not isdir(outdir):
  makedirs(outdir)

In [6]:
# create model
!python create-model.py eccv models/eccv-model-scracth

Creating model eccv
2022-01-11 17:59:40.490172: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Finished
Saving at models/eccv-model-scracth
/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [7]:
# file create_model.py tạo ra model rồi lưu vào 1 file
# hàm này load lại model đó vào đây
model, model_stride, xshape, yshape = load_network('models/eccv-model-scracth', dim) # xshape là (208,208,3), yshape là (13,13,10)
model.compile(loss=loss, optimizer=keras.optimizers.Adam(lr=0.001))  # compile model

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [8]:
print(xshape, yshape)

(208, 208, 3) (13, 13, 10)


In [9]:
print('Checking input directory...')
Data = load_Data(train_dir)
print('%d images with labels found' % len(Data))

Checking input directory...
206 images with labels found


In [10]:
dg = DataGenerator(	data=Data,
                        process_data_item_func=lambda x: process_data_item(
                            x, dim, model_stride),
                        xshape=xshape,  # (208,208,3)
                        yshape=(yshape[0], yshape[1],
                                yshape[2]),  # (13,13,10)
                        nthreads=2,
                        pool_size=1000,
                        min_nsamples=100)
dg.start()

In [11]:
Xtrain = np.empty((batch_size, dim, dim, 3),
                      dtype='single')  # (batch,208,208,3)
Ytrain = np.empty((batch_size, int(dim/model_stride),int(dim/model_stride), 2*4+2))  ###### fix # (batch,13,13,10)

In [12]:
val_dir = './samples/val_resized'
Val_Data = load_Data(val_dir)
Xval, Yval = process_Data(Val_Data)  #### phải label cho cả tập val nữa dcm

In [13]:
model_path_backup = '%s/%s_backup' % (outdir, netname)
model_path_final = '%s/%s_final' % (outdir, netname)

train_losses = []
val_losses = []

trainloss_txt = open('train_losses.txt', 'w')
valloss_txt = open('val_losses.txt', 'w')

In [ ]:
for it in range(iterations):
  print('Iter. %d (of %d)' % (it+1, iterations))

  Xtrain, Ytrain = dg.get_batch(batch_size)
  train_loss = model.train_on_batch(Xtrain, Ytrain)
  val_loss = model.test_on_batch(Xval, Yval, reset_metrics=True, return_dict=False)

  print('\tLoss: %f' % train_loss)
  print('\tVal Loss: %f' % val_loss)

  train_losses.append(train_loss)
  val_losses.append(val_loss)
  trainloss_txt.write(str(train_loss)+'\n')
  valloss_txt.write(str(val_loss)+'\n')

  # Save model every 1000 iterations
  if (it+1) % 1000 == 0:
      print('Saving model (%s)' % model_path_backup)
      save_model(model, model_path_backup)

print('Stopping data generator')
dg.stop()

trainloss_txt.close()
valloss_txt.close()

print('Saving model (%s)' % model_path_final)
save_model(model, model_path_final)

Iter. 1 (of 10000)
	Loss: 747.868286
	Val Loss: 160.394592
Iter. 2 (of 10000)
	Loss: 333.280853
	Val Loss: 159.694168
Iter. 3 (of 10000)
	Loss: 245.505829
	Val Loss: 151.908463
Iter. 4 (of 10000)
	Loss: 205.314423
	Val Loss: 144.496292
Iter. 5 (of 10000)
	Loss: 108.022598
	Val Loss: 133.873856
Iter. 6 (of 10000)
	Loss: 133.448364
	Val Loss: 140.718796
Iter. 7 (of 10000)
	Loss: 127.462646
	Val Loss: 288.431152
Iter. 8 (of 10000)
	Loss: 145.293610
	Val Loss: 659.893555
Iter. 9 (of 10000)
	Loss: 125.364716
	Val Loss: 977.039062
Iter. 10 (of 10000)
	Loss: 85.002113
	Val Loss: 1306.135986
Iter. 11 (of 10000)
	Loss: 87.648918
	Val Loss: 1690.532837
Iter. 12 (of 10000)
	Loss: 75.266800
	Val Loss: 2054.671387
Iter. 13 (of 10000)
	Loss: 76.152710
	Val Loss: 2556.853516
Iter. 14 (of 10000)
	Loss: 70.313492
	Val Loss: 3093.615723
Iter. 15 (of 10000)
	Loss: 78.359344
	Val Loss: 3688.708984
Iter. 16 (of 10000)
	Loss: 79.422943
	Val Loss: 4328.008789
Iter. 17 (of 10000)
	Loss: 97.947113
	Val Loss: 4

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Streaming output truncated to the last 5000 lines.
	Loss: 13.099957
	Val Loss: 34.761482
Iter. 2344 (of 10000)
	Loss: 11.122704
	Val Loss: 35.852146
Iter. 2345 (of 10000)
	Loss: 11.778547
	Val Loss: 33.206978
Iter. 2346 (of 10000)
	Loss: 13.253232
	Val Loss: 29.077282
Iter. 2347 (of 10000)
	Loss: 9.500584
	Val Loss: 27.604038
Iter. 2348 (of 10000)
	Loss: 8.601770
	Val Loss: 29.331165
Iter. 2349 (of 10000)
	Loss: 11.385113
	Val Loss: 34.850491
Iter. 2350 (of 10000)
	Loss: 10.126875
	Val Loss: 41.577122
Iter. 2351 (of 10000)
	Loss: 12.216204
	Val Loss: 47.494587
Iter. 2352 (of 10000)
	Loss: 9.742457
	Val Loss: 49.854664
Iter. 2353 (of 10000)
	Loss: 11.725124
	Val Loss: 51.614746
Iter. 2354 (of 10000)
	Loss: 11.665980
	Val Loss: 54.303070
Iter. 2355 (of 10000)
	Loss: 13.776867
	Val Loss: 57.546696
Iter. 2356 (of 10000)
	Loss: 12.571020
	Val Loss: 59.086990
Iter. 2357 (of 10000)
	Loss: 11.618059
	Val Loss: 59.929886
Iter. 2358 (of 10000)
	Loss: 10.028618
	Val Loss: 60.601208
Iter. 2359 (of